# 네이버 뉴스 수집 

In [53]:
import requests
from bs4 import BeautifulSoup

import os

In [54]:
# IT/과학 : 105, 경제 : 101, 사회 : 102, 생활/문화 : 103, 세계 : 104, 정치 : 100

CATEGORIES = [105,101,102,103,104,100] # 101에러

# 수집하고자 하는 카테고리의 번호 
CATEGORY = 103

# 수집하고자 하는 페이지 범위 
PAGES = range(1,50)

# 네이버 뉴스 URL
URL = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={category}#&date=%2000:00:00&page={page}'

# 접속 객체 생성
session = requests.Session()

# 접속객체에 부가정보(header) 삽입하기
session.headers.update({
    "Referer" : "",
    "User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
})


# 수집한 뉴스기사의 수를 저장할 변수
counter = 1

# 수집한 뉴스기사를 저장할 파일명
fileName = "./{myname}_naver_news_{category}_{counter}.txt"


# 저장할 폴더 경로 설정
folder_path = "news"  # 원하는 폴더 경로로 수정하세요

# 폴더가 없으면 폴더를 생성합니다
if not os.path.exists(folder_path):
    os.makedirs(folder_path)



In [55]:
# 주어진 카테고리에 대해 주어진 페이지 범위 만큼 뉴스 수집

for page in PAGES :
    targetUrl = URL.format(category=CATEGORY,page=page)
    print(targetUrl)

    # 생성한 접속 객체를 활용하여 뉴스 목록 페이지에 접속
    r = session.get(targetUrl)

    # 접속에 실패한 경우
    if r.status_code != 200 :
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)

    # 응답결과를 텍스트로 변환
    r.encoding = 'euc-kr'
    soup = BeautifulSoup(r.text)

    # 뉴스 기사 제목을 위한 CSS 선택자
    selector = ".sh_item .sh_text .sh_text_headline, #section_body ul li dl dt a"
    
    # 뉴스 기사 제목 추출
    headlines = soup.select(selector)

    # 각 페이지별로 추출된 뉴스기사의 본문 URL
    for h in headlines:
        href = h.attrs["href"]
        
        # 기사 본문에 대한 세부 내용 수집
        if href:
            print(href)
            bodyRes = session.get(href)
            
            # 접속에 실패한 경우
            if bodyRes.status_code != 200:
                # 에러코드와 에러메시지 출력
                msg = "[%d Error] %s 에러가 발생함" % (bodyRes.status_code, bodyRes.reason)
                # 에러를 강제로 생성시킴
                raise Exception(msg)
            
            bodyRes.encoding = "utf-8"
            bodySoup = BeautifulSoup(bodyRes.text)
            
            # 제목
            title = bodySoup.select("#title_area span")[0].text
            print(title)
            
            # 본문
            content = bodySoup.select("#dic_area")[0]
            
            # 본문 요소에 포함되어 있는 불필요 항목 제거
            for target in content.find_all("span", {"class": "end_photo_org"}):
                target.extract()
                
            for target in content.find_all("strong", {"class": "media_end_summary"}):
                target.extract()
                
            for target in content.find_all("table", {"class": "nbd_table"}):
                target.extract()
                
            for target in content.find_all("div", {"class": "highlightBlock"}):
                target.extract()
                
            for target in content.find_all("div", {"style": "display:none;"}):
                target.extract()
                
            for target in content.find_all("br"):
                target.replace_with("\n")
                
            contentBody = content.text.strip()
            
            # 파일 경로 설정
            file_name = fileName.format(myname="jisunlee", category=CATEGORY, counter=counter)
            file_path = os.path.join(folder_path, file_name)
            
            # 파일에 내용 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(title)
                f.write("\n\n")
                f.write(contentBody)

            # with open(fileName.format(myname="jisunlee", category=CATEGORY, counter=counter), "w", encoding="utf-8") as f:
            #     f.write(title)
            #     f.write("\n\n")
            #     f.write(contentBody)
                
            counter += 1
            
            print("%d news article crawled" % counter)

https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103#&date=%2000:00:00&page=1
https://n.news.naver.com/mnews/article/011/0004239368?sid=103
[속보] '가야고분군' 한국 16번째 세계유산 올랐다
2 news article crawled
https://n.news.naver.com/mnews/article/422/0000619935?sid=103
[날씨] 한낮 남부 30도 안팎 늦더위…전국 요란한 소나기
3 news article crawled
https://n.news.naver.com/mnews/article/009/0005188070?sid=103
제네시스, 전 차종 규격 ‘윈터타이어 패키지’ 판매
4 news article crawled
https://n.news.naver.com/mnews/article/009/0005187651?sid=103
"관광 아이디어로 지역소멸 막아요"
5 news article crawled
https://n.news.naver.com/mnews/article/079/0003813693?sid=103
설과 대보름·추석 등 우리 대표 명절, 국가무형문화재 된다
6 news article crawled
https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103#&date=%2000:00:00&page=2
https://n.news.naver.com/mnews/article/031/0000773699?sid=103
설과 추석 등 5개 명절 ‘국가유산’ 신규 지정한다
7 news article crawled
https://n.news.naver.com/mnews/article/421/0007056863?sid=103
'송학동고분군' 세계유산 등재 결실…고성군 소가야 역사도시 굳건히
8 news article crawled
https://n.news